In [ ]:
os.chdir('..')

In [ ]:

from model_config import *
from model_packages import *
from model_utils import *
import re

import add_counter_location_info 
# import add_census_features 
import add_census_features_2021
import add_green_blue_infrastructure
import add_land_classification
import add_land_habitat_data
import add_dog_occupancy
import add_weather
import add_POIs
import add_strava_data
import add_counter_data

In [ ]:
# Running each script  to generate static features data set for test sites
add_counter_location_info.process_test_sites_data('./scripts/test_sites_config.json')
add_census_features_2021.process_test_sites_data()
add_green_blue_infrastructure.process_test_sites_data()
add_land_classification.process_test_sites_data()
add_land_habitat_data.process_test_sites_data()
add_dog_occupancy.process_test_sites_data()
add_POIs.process_test_sites_data()

# access config file to loop through providers and datasets
config= add_counter_location_info.get_config_file_paths('./scripts/test_sites_config.json')


# process strava data for each provider
for provider, info in config.items():
    print(f'>>>>>>>>>>>>>{provider, info["strava_path"]}')
    add_strava_data.process_test_sites_data(provider, info['strava_path'])

# combine counts data for all providers
dfs=[]
for provider, info in config.items():
    df= pd.read_pickle(data_folder+f'test_sites/strava_data_{provider}.pkl')
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
combined_df['Date']=combined_df['Date'].astype(str)


# merge with weather features
add_weather.process_test_sites_data()
weather= pd.read_pickle(data_folder+'test_sites_weather_data.pkl')
weather['Date']=weather['Date'].astype(str)
dynamic= weather.merge(combined_df, on= ['Date', 'site'], how = 'right')

# merge with static features
static= pd.read_pickle(data_folder+'test_sites_static_data.pkl')
static_and_dynamic_features= dynamic[['site', 'Date', 'tavg', 'total_trip_count']].merge(static, left_on= 'site', right_on='counter', how= 'inner')

# save data of static and dynamic features for modelling
static_and_dynamic_features.to_pickle(data_folder+f'test_sites_static_and_dynamic_features_{bufr_zones_mrts}.pkl')